In [20]:
# imports
import os
import sys
sys.path.append('/mnt/0A2AAC152AABFBB7/CGE/luxgiant-clinical')
import luxgiant_clinical.MaleFemale as mf

import pandas as pd

In [21]:
folder_path = '/mnt/0A2AAC152AABFBB7/CGE/luxgiant-clinical/data'

df = pd.read_csv(
    os.path.join(folder_path, 'cleaned_file.csv'), 
    index_col=0, low_memory=False
)
matched_ids = pd.read_csv(
    os.path.join(folder_path, 'matched_ids.csv'), 
    index_col=0, low_memory=False
)
df = df.merge(matched_ids, on='participant_id')

In [22]:
# filter data to keep only patients with gender information
mask_patients= (df['Status']=='Patient')
mask_gender  = (~df['sex'].isnull()) 

df_cases = df[mask_patients & mask_gender].reset_index(drop=True)
del df

In [23]:
# selection of first group of variables 
variables_dict_1 = {
    "age"                       : "Age at Recruitment (years)",
    "age_at_onset"              : "Age at Onset (years)" ,
    "agecat_1"                  : "Onset <50 years" ,
    "agecat_2"                  : "Onset <40 years",
    "PD_duration"               : "Duration of Disease (years)",
    "years_of_education"        : "Years of Education",
    "bmi_comp"                  : "Body Mass Index",
    "work"                      : "Currently Employed",
    "nature_of_work___1"        : "Agriculture as a job",
    "over_your_lifetime_have_yo": "Life time direct exposure to pesticide/insecticide/fungicide",
    "during_your_lifetime_did_y": "Exposure to chemicals to kill insects in your life time",
    "in_your_lifetime_have_you" : "Smoked 100 or more cigarettes in lifetime",
    "in_your_lifetime_have"     : "Regular consumption of caffeinated drinks for >6months",
    "have_you_ever_had_a_head_i": "Head injury or concussion",

}
variables_1 = ['participant_id', 'sex'] + list(variables_dict_1.keys())

# statistical measures
stats_meas = {
    'mean'  : ["Age at Recruitment (years)", "Age at Onset (years)", "Body Mass Index"],
    'n'     : ["Onset <50 years", "Onset <40 years", "Currently Employed", "Agriculture as a job", 
               "Life time direct exposure to pesticide/insecticide/fungicide", "Exposure to chemicals to kill insects in your life time",
               "Smoked 100 or more cigarettes in lifetime", "Regular consumption of caffeinated drinks for >6months",
               "Head injury or concussion"],
    'median': ["Duration of Disease (years)", "Years of Education"]
}

In [24]:
df_cases_1 = df_cases[variables_1].copy()
df_cases_1 = df_cases_1.rename(columns=variables_dict_1)

df_cases_1["Onset <50 years"]     = df_cases_1["Onset <50 years"].map({'<50':1, '>=50':0})
df_cases_1["Onset <40 years"]     = df_cases_1["Onset <40 years"].map({'<40':1, '>=40':0})
df_cases_1["Currently Employed"]  = df_cases_1["Currently Employed"].map({'Yes':1, 'No':0})
df_cases_1["Agriculture as a job"]= df_cases_1["Agriculture as a job"].map({'Checked':1, 'Unchecked':0})

df_cases_1["Life time direct exposure to pesticide/insecticide/fungicide"] = \
    df_cases_1["Life time direct exposure to pesticide/insecticide/fungicide"]\
        .map({'Yes':1, 'No':0, 'Dont Know':0, 'Refused':0})
df_cases_1["Exposure to chemicals to kill insects in your life time"] =\
    df_cases_1["Exposure to chemicals to kill insects in your life time"]\
        .map({'Yes':1, 'No':0, 'Dont Know':0, 'Refused':0})
df_cases_1["Smoked 100 or more cigarettes in lifetime"] = \
    df_cases_1["Smoked 100 or more cigarettes in lifetime"]\
        .map({'Yes':1, 'No':0, 'Dont Know':0, 'Refused':0})
df_cases_1["Regular consumption of caffeinated drinks for >6months"] = \
    df_cases_1["Regular consumption of caffeinated drinks for >6months"]\
        .map({'Yes':1, 'No':0, 'Dont Know':0, 'Refused':0})
df_cases_1["Head injury or concussion"] = \
    df_cases_1["Head injury or concussion"].map({'Yes':1, 'No':0, 'Dont Know':0, 'Refused':0})

In [25]:
# dataframe for summary statistics

summary_cols = ['Variable', 'Statistical Measure', 'Male', 'Female', 'Available Sample for Analysis']
df_summary = pd.DataFrame(columns=summary_cols)

In [26]:
res1 = mf.mean_std(df_cases_1, stats_meas['mean'], grouping_by='sex')
df_summary1 = mf.summaryze_mean_std(df_summary, res1, stats_meas['mean'], 'Female', 'Male')
df_summary1 = df_summary1\
    .merge(
        mf.mean_std_simple(df_cases_1, features=stats_meas['mean'])
    )\
    .merge(
        mf.t_test_by_group(df_cases_1, stats_meas['mean'], group_var='sex'), on='Variable'
    )

In [27]:
res2 = mf.count_percent(df_cases_1, stats_meas['n'], 'sex')
df_summary2 = mf.summaryze_count_percent(df_summary, res2, stats_meas['n'], 'Female', 'Male')
df_summary2 = df_summary2\
    .merge(
        mf.count_simple(df_cases_1, features=stats_meas['n']), on='Variable'
    )\
    .merge(
        mf.chi_squared_tests(df_cases_1, stats_meas['n'], group_var='sex'), on='Variable'
    )

In [28]:
res3 = mf.median_iqr(df_cases_1, stats_meas['median'], 'sex')
df_summary3 = mf.summaryze_median_iqr(df_summary, res3, stats_meas['median'], 'Female', 'Male')
df_summary3 = df_summary3\
    .merge(
        mf.median_iqr_simple(df_cases_1, stats_meas['median']), on='Variable'
    )\
    .merge(
        mf.mann_whitney(df_cases_1, stats_meas['median'], 'sex'), on='Variable'
    )

In [29]:
ordered_cols = ['Variable', 'Statistical Measure', 'Male', 'Female', 'p-value', 'Total', 'Available Sample for Analysis']
df_summary = pd.concat(
    [df_summary1, df_summary3, df_summary2], axis=0,ignore_index=True
)
df_summary = df_summary[ordered_cols].copy()

In [30]:
# selection of second group of variables 
variables_dict_2 = {
    "medical_history_neurologic___7": "Stroke",
    "medical_history_cancer___1"    : "Melanoma" ,
    "medical_history_cancer___2"    : "Prostate Cancer" ,
    "medical_history_cancer___3"    : "Other Cancers",
    "medical_history_metabolic___1" : "Diabetes",
    "medical_history_metabolic___4" : "High Cholesterol",
    "medical_history_cardiovasc___2": "Hypertension",
    "medical_history_cardiovasc___3": "History of CAD",
    "medical_history_psychologi___1": "History of Anxiety",
    "family_member_diagnosed_wi"    : "Family History of PD",
    "family_member_diagnosed_wi_2"  : "Family History of Tremor",
    "family_member_diagnosed"       : "Family History of Dementia"
}
variables_2 = ['participant_id', 'sex'] + list(variables_dict_2.keys())

# statistical measures
stats_meas2 = {
    'n'     : ["Stroke", "Melanoma", "Prostate Cancer", "Other Cancers", "Diabetes", "High Cholesterol", "Hypertension",
               "History of CAD", "History of Anxiety", "Family History of PD", "Family History of Tremor", "Family History of Dementia"]
}

In [31]:
df_cases_2 = df_cases[variables_2].copy()
df_cases_2 = df_cases_2.rename(columns=variables_dict_2)

df_cases_2["Stroke"]                    = df_cases_2["Stroke"].map({"Checked":1, "Unchecked":0})
df_cases_2["Melanoma"]                  = df_cases_2["Melanoma"].map({"Checked":1, "Unchecked":0})
df_cases_2["Prostate Cancer"]           = df_cases_2["Prostate Cancer"].map({"Checked":1, "Unchecked":0})
df_cases_2["Other Cancers"]             = df_cases_2["Other Cancers"].map({"Checked":1, "Unchecked":0})
df_cases_2["Diabetes"]                  = df_cases_2["Diabetes"].map({"Checked":1, "Unchecked":0})
df_cases_2["High Cholesterol"]          = df_cases_2["High Cholesterol"].map({"Checked":1, "Unchecked":0})
df_cases_2["Hypertension"]              = df_cases_2["Hypertension"].map({"Checked":1, "Unchecked":0})
df_cases_2["History of CAD"]            = df_cases_2["History of CAD"].map({"Checked":1, "Unchecked":0})
df_cases_2["History of Anxiety"]        = df_cases_2["History of Anxiety"].map({"Checked":1, "Unchecked":0})
df_cases_2["Family History of PD"]      = df_cases_2["Family History of PD"].map({"Yes":1, "No":0})
df_cases_2["Family History of Tremor"]  = df_cases_2["Family History of Tremor"].map({"Yes":1, "No":0})
df_cases_2["Family History of Dementia"]= df_cases_2["Family History of Dementia"].map({"Yes":1, "No":0})

In [32]:
# dataframe for summary statistics

summary_cols = ['Variable', 'Statistical Measure', 'Male', 'Female', 'Available Sample for Analysis']
df_summary_med = pd.DataFrame(columns=summary_cols)

In [33]:
res4 = mf.count_percent(df_cases_2, stats_meas2['n'], 'sex')
df_summary_med = mf.summaryze_count_percent(df_summary_med, res4, stats_meas2['n'], 'Female', 'Male')
df_summary_med = df_summary_med\
    .merge(
        mf.count_simple(df_cases_2, features=stats_meas2['n']), on='Variable'
    )\
    .merge(
        mf.chi_squared_tests(df_cases_2, stats_meas2['n'], group_var='sex'), on='Variable'
    )
df_summary_med = df_summary_med[ordered_cols].copy()

df_summary_med = pd.concat([
    pd.DataFrame(data=[["Medical History", "", "", "", "", "", ""]], columns=ordered_cols), df_summary_med
],
ignore_index=True, axis=0
)

In [34]:
df = pd.concat([df_summary, df_summary_med], axis=0, ignore_index=True)
df.to_csv('/mnt/0A2AAC152AABFBB7/CGE/luxgiant-clinical/data/Table_4.csv', index=False)
df

,Variable,Statistical Measure,Male,Female,p-value,Total,Available Sample for Analysis
0,Age at Recruitment (years),mean (SD),56.29 (12.31),57.42 (11.43),0.0087,56.71 (12.0),3320
1,Age at Onset (years),mean (SD),49.45 (12.85),50.81 (12.08),0.0027,49.96 (12.58),3320
2,Body Mass Index,mean (SD),24.28 (3.52),23.97 (3.88),0.0265,24.17 (3.65),3029
3,Duration of Disease (years),median (IQR),5.0 (3.0 - 10.0),5.0 (3.0 - 10.0),0.1705,5.0 (3.0 - 10.0),3320
4,Years of Education,median (IQR),12.0 (10.0 - 15.0),10.0 (5.0 - 15.0),p<0.001,12.0 (8.0 - 15.0),3217
5,Onset <50 years,n (%),1104 (52.82),538 (43.74),p<0.001,1642 (49.46),3320
6,Onset <40 years,n (%),497 (23.78),236 (19.19),0.0024,733 (22.08),3320
7,Currently Employed,n (%),1337 (65.44),564 (46.77),p<0.001,1901 (57.26),3249
8,Agriculture as a job,n (%),370 (17.7),132 (10.73),p<0.001,502 (15.12),3320
9,Life time direct exposure to pesticide/insecti...,n (%),452 (21.63),161 (13.09),p<0.001,613 (18.46),3320


In [35]:
df_1 = pd.read_csv('/mnt/0A2AAC152AABFBB7/CGE/luxgiant-clinical/data/Table_1.csv')
df_1 = df_1.drop(columns='Statistical Measure')

In [36]:
multi_index_4 = pd.MultiIndex.from_tuples(
    [('', 'Variables'), ('', 'Statistical Measure'),
     ('Whole PD Cohort', 'Male'), 
     ('Whole PD Cohort', 'Female'),
     ('Whole PD Cohort', 'p-value'),
     ('Whole PD Cohort', 'Total'),
     ('Whole PD Cohort', 'Available Samples for Analysis')]
)
df.columns = multi_index_4
df

\
                                            Variables Statistical Measure   
0                          Age at Recruitment (years)           mean (SD)   
1                                Age at Onset (years)           mean (SD)   
2                                     Body Mass Index           mean (SD)   
3                         Duration of Disease (years)        median (IQR)   
4                                  Years of Education        median (IQR)   
5                                     Onset <50 years               n (%)   
6                                     Onset <40 years               n (%)   
7                                  Currently Employed               n (%)   
8                                Agriculture as a job               n (%)   
9   Life time direct exposure to pesticide/insecti...               n (%)   
10  Exposure to chemicals to kill insects in your ...               n (%)   
11          Smoked 100 or more cigarettes in lifetime               n (%)   
12  Regular consumption of caffeinated drinks for ...               n (%)   
13                          Head injury or concussion               n (%)   
14                                    Medical History                       
15                                             Stroke               n (%)   
16                                           Melanoma               n (%)   
17                                    Prostate Cancer               n (%)   
18                                      Other Cancers               n (%)   
19                                           Diabetes               n (%)   
20                                   High Cholesterol               n (%)   
21                                       Hypertension               n (%)   
22                                     History of CAD               n (%)   
23                                 History of Anxiety               n (%)   
24                               Family History of PD               n (%)   
25                           Family History of Tremor               n (%)   
26                         Family History of Dementia               n (%)   

       Whole PD Cohort                                                 \
                  Male             Female  p-value              Total   
0        56.29 (12.31)      57.42 (11.43)   0.0087       56.71 (12.0)   
1        49.45 (12.85)      50.81 (12.08)   0.0027      49.96 (12.58)   
2         24.28 (3.52)       23.97 (3.88)   0.0265       24.17 (3.65)   
3     5.0 (3.0 - 10.0)   5.0 (3.0 - 10.0)   0.1705   5.0 (3.0 - 10.0)   
4   12.0 (10.0 - 15.0)  10.0 (5.0 - 15.0)  p<0.001  12.0 (8.0 - 15.0)   
5         1104 (52.82)        538 (43.74)  p<0.001       1642 (49.46)   
6          497 (23.78)        236 (19.19)   0.0024        733 (22.08)   
7         1337 (65.44)        564 (46.77)  p<0.001       1901 (57.26)   
8           370 (17.7)        132 (10.73)  p<0.001        502 (15.12)   
9          452 (21.63)        161 (13.09)  p<0.001        613 (18.46)   
10         479 (22.92)         178 (14.5)  p<0.001        657 (19.79)   
11         314 (15.04)           16 (1.3)  p<0.001         330 (9.94)   
12         530 (25.37)        233 (18.97)  p<0.001        763 (22.98)   
13           194 (9.4)          63 (5.18)  p<0.001         257 (7.74)   
14                                                                      
15           13 (0.62)          13 (1.06)   0.2424          26 (0.78)   
16            1 (0.05)            0 (0.0)      1.0           1 (0.03)   
17            1 (0.05)            0 (0.0)      1.0           1 (0.03)   
18           10 (0.48)          12 (0.98)   0.1379          22 (0.66)   
19         396 (18.95)        226 (18.37)   0.7167        622 (18.73)   
20           53 (2.54)          87 (7.07)  p<0.001         140 (4.22)   
21         500 (23.92)        332 (26.99)   0.0538        832 (25.06)   
22           29 (1.39)          11 (0.89)   0.2742           40 (1.2)   
23          118 

In [37]:
multi_index_1 = pd.MultiIndex.from_tuples(
    [('', 'Variables'),
     ('Sex Matched Cohort', 'Male'), 
     ('Sex Matched Cohort', 'Female'),
     ('Sex Matched Cohort', 'p-value'),
     ('Sex Matched Cohort', 'Total'),
     ('Sex Matched Cohort', 'Available Samples for Analysis')]
)
df_1.columns = multi_index_1
df_1

Sex Matched Cohort  \
                                            Variables                Male   
0                          Age at Recruitment (years)       60.89 (11.43)   
1                                Age at Onset (years)       54.61 (11.94)   
2                                     Body Mass Index        24.29 (3.47)   
3                         Duration of Disease (years)     5.0 (2.0 - 9.0)   
4                                  Years of Education  12.0 (10.0 - 15.0)   
5                                     Onset <50 years        1591 (31.42)   
6                                     Onset <40 years         565 (11.16)   
7                                  Currently Employed        2816 (54.74)   
8                                Agriculture as a job         906 (16.88)   
9   Life time direct exposure to pesticide/insecti...        1154 (21.68)   
10  Exposure to chemicals to kill insects in your ...         1139 (21.4)   
11          Smoked 100 or more cigarettes in lifetime         869 (16.33)   
12  Regular consumption of caffeinated drinks for ...        1364 (25.64)   
13                          Head injury or concussion          408 (7.84)   
14                                    Medical History                 NaN   
15                                             Stroke           50 (0.93)   
16                                           Melanoma            4 (0.07)   
17                                    Prostate Cancer           12 (0.22)   
18                                      Other Cancers           31 (0.58)   
19                                           Diabetes        1145 (21.33)   
20                                   High Cholesterol           96 (1.79)   
21                                       Hypertension        1420 (26.46)   
22                                     History of CAD          149 (2.78)   
23                                 History of Anxiety           279 (5.2)   
24                               Family History of PD          486 (9.37)   
25                           Family History of Tremor          442 (8.89)   
26                         Family History of Dementia           86 (1.73)   

                                                   \
               Female  p-value              Total   
0       59.69 (11.04)  p<0.001       60.5 (11.32)   
1       53.46 (11.51)  p<0.001      54.24 (11.81)   
2         24.17 (4.0)   0.1928       24.25 (3.64)   
3     5.0 (2.0 - 9.0)   0.6474    5.0 (2.0 - 9.0)   
4   10.0 (5.0 - 15.0)  p<0.001  12.0 (9.0 - 15.0)   
5         858 (35.71)  p<0.001       2449 (30.93)   
6         268 (11.15)      1.0        833 (10.52)   
7         943 (38.63)  p<0.001       3759 (47.47)   
8          246 (9.64)  p<0.001       1152 (14.55)   
9         382 (15.04)  p<0.001        1536 (19.4)   
10        396 (15.61)  p<0.001       1535 (19.39)   
11           33 (1.3)  p<0.001        902 (11.39)   
12        570 (22.47)   0.0025       1934 (24.43)   
13         138 (5.54)  p<0.001          546 (6.9)   
14                NaN      NaN                NaN   
15           23 (0.9)   0.9962          73 (0.92)   
16            0 (0.0)   0.3986           4 (0.05)   
17           1 (0.04)   0.1103          13 (0.16)   
18          32 (1.25)   0.0024           63 (0.8)   
19        427 (16.74)  p<0.001       1572 (19.85)   
20         142 (5.57)  p<0.001         238 (3.01)   
21        678 (26.58)   0.9317        2098 (26.5)   
22          31 (1.22)  p<0.001         180 (2.27)   
23         123 (4.82)   0.5099         402 (5.08)   
24         223 (9.02)    0.653         709 (8.95)   
25         181 (7.65)   0.0839         623 (7.87)   
26          38 (1.62)   0.7949         124 (1.57)   

                                   
   Available Samples for Analysis  
0                          7905.0  
1                          7467.0  
2                          6904.0  
3                          7472.0  
4                          7365.0  
5                          7467

In [40]:
df_mrg = pd.merge(df, df_1, on=[('', 'Variables')])

In [41]:
df_mrg.to_csv(
    '/mnt/0A2AAC152AABFBB7/CGE/luxgiant-clinical/data/Table_1_4.csv'
)